# Lowest Risk Aircraft for Commercial and Private Use - Analysis


## Overview

This project analyzes the resource needs of the [Austin Animal Center](https://www.austintexas.gov/department/aac) (AAC), which shelters 16,000 animals annually with a [No Kill policy](https://www.austintexas.gov/blog/no-kill-austin). Descriptive analysis of animal intake and outcome data shows that some animals require extended stays and that the number of sheltered animals varies seasonally. The Austin Animal Center can use this analysis to adjust outreach, hiring, and space utilization to improve resource allocation.

## Business Problem

Your company is expanding in to new industries to diversify its portfolio. Specifically, they are interested in purchasing and operating airplanes for commercial and private enterprises, but do not know anything about the potential risks of aircraft. You are charged with determining which aircraft are the lowest risk for the company to start this new business endeavor. You must then translate your findings into actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase.

## Business Understanding

Determine the lowest risk aircraft for commercial and private use that a company venturing into (aircraft industry for the first time) can use and translate the findings in to purchasing decisions
### Objectives
We want an aircraft that has,

- Minimized safety risks to avoid accidents, being sued and reputation damage (Human safety risk)
- Low asset preservation risk, ensuring long-term value and reliability.
- Minimal depreciation, minimal downtime, high fuel efficiency, easily available spare parts to build company's trust on the aircraft dependability (operational risk)

## Data Understanding

I will use the National Transportation safety board dataset from 1962 to 2023 about civil aviation incident in U.S and International waters.
I need to understand the structure of the data. The columns likely include information about the aircraft, the accident, injuries, fatalities, causes. 
Let's see:

In [6]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('data/AviationData.csv', encoding = 'latin-1', low_memory = False)
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [9]:
# number of rows and columns
# type of data in each column

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

There are 88889 rows and 31 columns in this dataset

In [13]:
# number of missing values in each columns
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publication.Date          13771
dtype: i

`Event.id`, `Investigation.type`, `Accident.Number`, `Event.Date` are the likely candidates of being the unique id/s of this dataset because they have no missing values(also they are the first columns of this dataset)

In [16]:
# Basic Statistics of numerical columns
df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


`Number.of.Engines` is meaningless based on our problem

## Data Preparation

### Columns To Use
Because this problem focuses on the lowest risk aircraft, some of the columns are unnecessary and meaningless to us so I will drop them.
Columns tha I have found to be meaningful to us(I googled search all the columns and filtered them according to the problem)

- `Make` and `Model`: Identify the aircraft.
- `Event.Date`: For temporal trends (convert to year/decade).
- `Injury.Severity`: Categorical (e.g., Fatal, Serious, Minor, None).
- `Aircraft.damage`: Extent of damage (e.g., Destroyed, Substantial, Minor).
- `Total.Fatal.Injuries`, `Total.Serious.Injuries`, `Total.Minor.Injuries`, `Total.Uninjured`: Quantify the severity.
- `Broad.phase.of.flight`: When the accident occurred (e.g., Takeoff, Landing, Cruise).
- `Weather.Condition`: Factor in weather (e.g., VMC, IMC).
- `Aircraft.Category`: To filter for airplane (exclude balloons, gliders, etc.).
- `Purpose.of.flight`: Commercial vs. private? (e.g., Personal, Instructional, Aerial Work, Public Transport)

### Data Cleaning



- For `Make` and `Model`: if missing, we drop the rows because we cannot associate with an aircraft model.

In [23]:
df['Make'].isna().sum()
# small number of NaN, dropping rows will be the best technique

63

In [25]:
# Doing the same for model
df['Model'].isna().sum()

92

In [27]:
df.dropna(subset = ['Make', 'Model'], inplace = True)

In [29]:
#confirming
df[['Make', 'Model']].isna().sum()

Make     0
Model    0
dtype: int64

For `Event.date` there are no missing values

In [32]:
df['Event.Date'].isna().sum()
# no missing values

0

In [98]:
df['Event.Date'].dtype

dtype('O')

- For `Injury.Severity`: If missing, we can infer from the injury columns. For example, if `Total.Fatal.Injuries` > 0, then set to 'Fatal', if `Total.Serious.Injuries` > 0 set to 'serious' and so on. If all the columns are equal to 0 set the value to 'unavailable'

In [37]:
df.loc[df['Injury.Severity'].isna(), ['Injury.Severity', 'Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].sample(n = 20)

,Injury.Severity,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
80906,NaN,0.0,0.0,0.0,0.0
72014,NaN,0.0,0.0,0.0,0.0
87130,NaN,0.0,0.0,0.0,0.0
75861,NaN,0.0,0.0,0.0,0.0
78340,NaN,0.0,0.0,0.0,0.0
85032,NaN,0.0,0.0,0.0,0.0
79620,NaN,0.0,0.0,0.0,0.0
84127,NaN,0.0,0.0,0.0,0.0
86773,NaN,0.0,0.0,0.0,0.0
67452,NaN,0.0,0.0,0.0,0.0


I have seen that if `injury.Severity` = NaN then the other columns = 0, so I will set `injury.Severity` to 'Unavailable'

In [40]:
df['Injury.Severity'].value_counts().head(12)

Injury.Severity
Non-Fatal      67305
Fatal(1)        6159
Fatal           5248
Fatal(2)        3703
Incident        2214
Fatal(3)        1145
Fatal(4)         812
Fatal(5)         235
Minor            218
Serious          173
Fatal(6)         161
Unavailable       95
Name: count, dtype: int64

In [42]:
# replacing 'NaN' with 'Unavailable'
df.loc[df['Injury.Severity'].isna(), 'Injury.Severity'] = 'Unavailable'

In [44]:
df['Injury.Severity'].isna().sum()

0

#### `Aircraft.damage`, `Weather.Condition` and `Purpose.of.flight` Data Cleaning

- For `Aircraft.damage`, `Weather.Condition` and `Purpose.of.flight`  I will replace 'NaN' with 'UNKNOWN' to preserve the data, I have observed that each column has a small percentage of missing data but if I was to drop all together, a huge chunk of the data will be lost

In [48]:
# printing the percentage of NaN values in this columns Aircraft.damage, Weather.Condition and Purpose.of.flight
print(df['Aircraft.damage'].isna().value_counts(normalize = True)* 100)
print(df['Weather.Condition'].isna().value_counts(normalize = True)* 100)
print(df['Purpose.of.flight'].isna().value_counts(normalize = True)* 100)

Aircraft.damage
False    96.427002
True      3.572998
Name: proportion, dtype: float64
Weather.Condition
False    94.999831
True      5.000169
Name: proportion, dtype: float64
Purpose.of.flight
False    93.086047
True      6.913953
Name: proportion, dtype: float64


 ##### `weather.condition` replacing NaN with UNK(unknown)

In [51]:
# printing unique values of weather.condition
df['Weather.Condition'].value_counts()
# UNK and unk represents unknown

Weather.Condition
VMC    77251
IMC     5971
UNK      854
Unk      262
Name: count, dtype: int64

In [53]:
# mapping Unk to UNK(for consistency) and filling missing values 'NaN' with UNK(unknown)
unkToUNK = {'Unk' : 'UNK', 'IMC' : 'IMC', 'UNK' : 'UNK', 'VMC' : 'VMC'}
df['Weather.Condition'] = df['Weather.Condition'].map(unkToUNK).fillna('UNK')

In [55]:
# printing unique values of weather.condition to see the results
df['Weather.Condition'].value_counts()

Weather.Condition
VMC    77251
IMC     5971
UNK     5555
Name: count, dtype: int64

##### `Purpose.of.flight` replacing 'NaN' with 'Unknown'

In [58]:
#printing each unique value and it's count
df['Purpose.of.flight'].value_counts()

Purpose.of.flight
Personal                     49413
Instructional                10599
Unknown                       6787
Aerial Application            4710
Business                      4016
Positioning                   1645
Other Work Use                1264
Ferry                          812
Aerial Observation             794
Public Aircraft                720
Executive/corporate            553
Flight Test                    404
Skydiving                      182
External Load                  123
Public Aircraft - Federal      105
Banner Tow                     101
Air Race show                   99
Public Aircraft - Local         74
Public Aircraft - State         64
Air Race/show                   59
Glider Tow                      53
Firefighting                    40
Air Drop                        11
ASHO                             6
PUBS                             4
PUBL                             1
Name: count, dtype: int64

In [60]:
df['Purpose.of.flight'] = df['Purpose.of.flight'].fillna('Unknown')

In [62]:
df['Purpose.of.flight'].value_counts()

Purpose.of.flight
Personal                     49413
Unknown                      12925
Instructional                10599
Aerial Application            4710
Business                      4016
Positioning                   1645
Other Work Use                1264
Ferry                          812
Aerial Observation             794
Public Aircraft                720
Executive/corporate            553
Flight Test                    404
Skydiving                      182
External Load                  123
Public Aircraft - Federal      105
Banner Tow                     101
Air Race show                   99
Public Aircraft - Local         74
Public Aircraft - State         64
Air Race/show                   59
Glider Tow                      53
Firefighting                    40
Air Drop                        11
ASHO                             6
PUBS                             4
PUBL                             1
Name: count, dtype: int64

#### `Aircraft.damage` replacing 'NaN' with 'Unknown'

In [65]:
df['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    64097
Destroyed      18597
Minor           2792
Unknown          119
Name: count, dtype: int64

In [67]:
# replacing 'NaN' with 'Unknown'
df['Aircraft.damage'] = df['Aircraft.damage'].fillna('Unknown')

In [69]:
df['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    64097
Destroyed      18597
Unknown         3291
Minor           2792
Name: count, dtype: int64

`Broad.phase.of.flight`

#### `Aircraft.Category` Data Cleaning

Missing data is around 63.6%, this is a large % we cannot drop the rows, also we cannot drop the column as this column is very critical  for filtering to "Airplane" as per our business focus, But looking on the business problem the company wants an 'aircraft' not specifically an 'airplane'
- Also on the data (non-null) that we have Airplane makes 85%, so it is safe to assume that the others most of them will be Airplane/ another assumption is that airplanes are well documented than the other categories, so I will drop the column(won't use it)

#### `Total.Fatal.Injuries`, `Total.Serious.Injuries`, `Total.Minor.Injuries`, `Total.Uninjured` Data Cleaning

I will handle the missing data by assigning it 0 (assuming No injuries if missing)

In [76]:
# printing percentage of missing values of this columns: Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries, Total.Uninjured
print(df['Total.Fatal.Injuries'].isna().value_counts(normalize = True) * 100)
print(df['Total.Serious.Injuries'].isna().value_counts(normalize = True) * 100)
print(df['Total.Minor.Injuries'].isna().value_counts(normalize = True)* 100)
print(df['Total.Uninjured'].isna().value_counts(normalize = True)* 100)

Total.Fatal.Injuries
False    87.174606
True     12.825394
Name: proportion, dtype: float64
Total.Serious.Injuries
False    85.931041
True     14.068959
Name: proportion, dtype: float64
Total.Minor.Injuries
False    86.579857
True     13.420143
Name: proportion, dtype: float64
Total.Uninjured
False    93.357514
True      6.642486
Name: proportion, dtype: float64


In [78]:
# Replacing NaN with 0 for each column
df['Total.Fatal.Injuries'] = df['Total.Fatal.Injuries'].fillna(0)
df['Total.Serious.Injuries'] = df['Total.Serious.Injuries'].fillna(0)
df['Total.Minor.Injuries'] = df['Total.Minor.Injuries'].fillna(0)
df['Total.Uninjured'] = df['Total.Uninjured'].fillna(0)

In [80]:
# printing percentage of missing values of this columns: Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries, Total.Uninjured
# After replacing NaN with 0
print(df['Total.Fatal.Injuries'].isna().value_counts(normalize = True) * 100)
print(df['Total.Serious.Injuries'].isna().value_counts(normalize = True) * 100)
print(df['Total.Minor.Injuries'].isna().value_counts(normalize = True)* 100)
print(df['Total.Uninjured'].isna().value_counts(normalize = True)* 100)

Total.Fatal.Injuries
False    100.0
Name: proportion, dtype: float64
Total.Serious.Injuries
False    100.0
Name: proportion, dtype: float64
Total.Minor.Injuries
False    100.0
Name: proportion, dtype: float64
Total.Uninjured
False    100.0
Name: proportion, dtype: float64


### Handling duplicates

There are no duplicates in this dataset

In [84]:
df.duplicated().value_counts()

False    88777
Name: count, dtype: int64

### Aggregation of data

### New Features

In [87]:
# purpose.of.flight renamed and assigned to a new column 'purpose.grouped' for easy understanding 
purpose_business_map = {
    # COMMERCIAL (Revenue-generating operations)
    'Aerial Application': 'Commercial',  
    'Banner Tow': 'Commercial',          
    'Business': 'Commercial',             
    'Executive/corporate': 'Commercial',  
    'Ferry': 'Commercial',                
    'Other Work Use': 'Commercial',       
    'Positioning': 'Commercial',          
    'Aerial Observation': 'Commercial',   

    # PRIVATE (Non-commercial personal/educational)
    'Personal': 'Private',                
    'Instructional': 'Private',          

    # GOVERNMENT (Public service)
    'Firefighting': 'Government',         
    'Public Aircraft': 'Government',      
    'Public Aircraft - Federal': 'Government',
    'Public Aircraft - Local': 'Government',
    'Public Aircraft - State': 'Government',

    # UTILITY (Specialized commercial work)
    'Air Drop': 'Utility',                                   
    'External Load': 'Utility',           

    # OTHER
    'Air Race show': 'Other',            
    'Air Race/show': 'Other',
    'Flight Test': 'Other',               
    'Glider Tow': 'Other',                
    'Skydiving': 'Other',                 

    #Unknown
    'PUBS': 'Unknown',                  
    'PUBL': 'Unknown',                  
    'Unknown': 'Unknown',
    'ASHO': 'Unknown'
}

df['Purpose.grouped'] = df['Purpose.of.flight'].map(purpose_business_map)
df['Purpose.grouped'].value_counts()

Purpose.grouped
Private       60012
Commercial    13895
Unknown       12936
Government     1003
Other           797
Utility         134
Name: count, dtype: int64

- For `Event.date` no missing values, but the month and day are of no value to us so I will extract the year and pu it in a new column `year`

In [89]:
# This code gets the year of each value in date column
df['Year'] = df['Event.Date'].map(lambda x: x.split('-')[0])

In [91]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date', 'Purpose.grouped', 'Year'],
      dtype='object')

## Data Analysis

In [166]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

## Conclusions

This analysis leads to three recommendations for improving operations of the Austin Animal Center:

- **Engage in targeted outreach campaigns for dogs that have been sheltered at AAC for more than 30 days.** While most dogs will have been placed after 30 days, this may help reduce the number of dogs that end up having extended stays, potentially requiring many more months of care.
- **Reduce current spending until the numbers of intakes and sheltered animals return to normal.** Given the reduced activity during this period, AAC should consider ways to temporarily reduce costs by changing space utilization or staffing.
- **Hire seasonal staff and rent temporary space for May through December.** To accommodate the high volume of intakes and number of sheltered animals in the spring and fall, AAC should leverage seasonal resources, rather than full-year ones. This will allow AAC to cut back on expenditures during the months when there is lower

### Next Steps

Further analyses could yield additional insights to further improve operations at AAC:

- **Better prediction of animals that are likely to have long stays.** This modeling could use already available data, such as breed and intake condition.
- **Model need for medical support.** This modeling could predict the need for specialized personnel to address animals' medical needs, including neutering, using intake condition and sex data.
- **Predicting undesirable outcomes.** This modeling could identify animals that are more likely to have undesirable outcomes (e.g. Euthanasia) for targeted medical support or outreach.